In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_cirrhosis"
cohort = "GSE285291"

# Input paths
in_trait_dir = "../../input/GEO/Liver_cirrhosis"
in_cohort_dir = "../../input/GEO/Liver_cirrhosis/GSE285291"

# Output paths
out_data_file = "../../output/preprocess/Liver_cirrhosis/GSE285291.csv"
out_gene_data_file = "../../output/preprocess/Liver_cirrhosis/gene_data/GSE285291.csv"
out_clinical_data_file = "../../output/preprocess/Liver_cirrhosis/clinical_data/GSE285291.csv"
json_path = "../../output/preprocess/Liver_cirrhosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Intestinal Mucosal Mitochondrial Oxidative Phosphorylation worsens with Cirrhosis Progression and is ameliorated with FMT"
!Series_summary	"Intestinal barrier dysfunction, driven by increased oxidative phosphorylation (OXPHOS)   activity that leads to tissue hypoxia, contributes to the progression of cirrhosis, particularly   impacting the upper intestine. This study explores the interplay between intestinal OXPHOS,   gut microbiota changes, and the effects of fecal microbiota transplant (FMT) in cirrhotic   patients. We investigated 32 age-matched men across three groups: healthy controls,   compensated cirrhosis, and decompensated cirrhosis. Each underwent endoscopy with   duodenal and ascending colon biopsies. Subsequently, in a follow-up study, nine patients with   hepatic encephalopathy, previously enrolled in a randomized controlled trial for FMT capsules,   underwent repeat pre and post-FMT upper endoscopy. Our bioinformatics analysis highl

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
is_gene_available = True  # Based on the background, this is a transcriptomic study of OXPHOS genes

# 2.1 Data Availability
# For cirrhosis trait - row 1 has "status" which indicates cirrhosis status
trait_row = 1

# Age is not mentioned in sample characteristics, not available
age_row = None

# Gender is not explicitly available, but the background mentions "32 age-matched men"
# Since all subjects are male, gender is a constant feature and not useful for our analysis
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert cirrhosis status to binary: Control=0, Compensated/Decompensated=1."""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
        if value == "Control":
            return 0
        elif value in ["Compensated", "Decompensated"]:
            return 1
    return None

def convert_age(value):
    """Not used as age data is not available."""
    return None

def convert_gender(value):
    """Not used as gender data is not available."""
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# For this step, we need to first check if clinical_data is available
# Since we don't have a clinical_data.csv file, we'll skip this substep
print("Clinical data file not found. Skipping clinical feature extraction.")


Clinical data file not found. Skipping clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Liver_cirrhosis/GSE285291/GSE285291_series_matrix.txt.gz
Gene data shape: (803, 53)
First 20 gene/probe identifiers:
Index(['A2M', 'AADAT', 'AANAT', 'ABCB7', 'ABL1', 'ACAA2', 'ACACA', 'ACACB',
       'ACAD11', 'ACADL', 'ACADM', 'ACADVL', 'ACAP2', 'ACAT1', 'ACAT2',
       'ACMSD', 'ACO2', 'ACOT12', 'ACOX1', 'ACSF3'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be standard human gene symbols already.
# They follow the standard HGNC naming conventions for human genes.
# For example:
# A2M = Alpha-2-Macroglobulin
# AADAT = Aminoadipate Aminotransferase
# ACACA = Acetyl-CoA Carboxylase Alpha
# ACO2 = Aconitase 2

# The identifiers are already in the correct format and don't need mapping
requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Check if trait_row is None (indicating no clinical data is available)
if trait_row is None:
    print("No clinical data available for this dataset, skipping clinical data processing.")
    
    # Validate and save cohort information with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use this data without clinical features
        df=pd.DataFrame(),  # Empty DataFrame since we have no linked data
        note="Dataset contains gene expression data from cell lines with HCV infection, which is not appropriate for liver cirrhosis trait analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical data. No linked data file saved.")
else:
    # If clinical data is available, proceed with the linking and processing
    # 2. Link the clinical and genetic data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty after missing value handling")

    # 5. Validate and save cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note="Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape before normalization: (803, 53)
Gene data shape after normalization: (800, 53)
Normalized gene expression data saved to ../../output/preprocess/Liver_cirrhosis/gene_data/GSE285291.csv
Selected clinical data shape: (1, 53)
Clinical data preview:
                 GSM8700031  GSM8700032  GSM8700033  GSM8700034  GSM8700035  \
Liver_cirrhosis         1.0         1.0         0.0         1.0         1.0   

                 GSM8700036  GSM8700037  GSM8700038  GSM8700039  GSM8700040  \
Liver_cirrhosis         1.0         1.0         1.0         1.0         0.0   

                 ...  GSM8700074  GSM8700075  GSM8700076  GSM8700077  \
Liver_cirrhosis  ...         1.0         1.0         1.0         1.0   

                 GSM8700078  GSM8700079  GSM8700080  GSM8700081  GSM8700082  \
Liver_cirrhosis         1.0         1.0         1.0         1.0         1.0   

                 GSM8700083  
Liver_cirrhosis         1.0  

[1 rows x 53 columns]
Linked data shape before processin

Linked data saved to ../../output/preprocess/Liver_cirrhosis/GSE285291.csv
